In [1]:
"""
This extracts data from the dataset files since they are too big to all fit in memory.

Assumes the original dataset folders are located in the same directory in their original form.

Author: Wesley
"""
import numpy as np
import pandas as pd
import dask.dataframe as dd # get dask with: pip install "dask[complete]"
from os import listdir
from os.path import isfile, join
import os

Since the original dataset files are too big to be loaded into memory, and since the benign cases are spread out across all the parts, we extract what we need from all of them to prepare our extract.

In [4]:
# Assumes we've placed program in root of dataset folder, where both days are subdirectories.
path1 = os.getcwd() + "/01-12"
path2 = os.getcwd() + "/03-11"

In [5]:
# Get list of files in each directory.
fileList = [path1 + '/' + f for f in listdir(path1) if isfile(join(path1, f))]
fileList2 = [path2 + '/' + f for f in listdir(path2) if isfile(join(path2, f))]

In [7]:
"""
This function extracts an equal amount of benign and attack data for each day and forms the train/test sets.

Needs Dask due to how large many of the datasets are.
"""
def extract(fileList):

    print("Beginning extraction of benign data...")
    # First pass aggregates BENIGN data.
    i = 0
    outFrame = None
    for f in fileList:
        df = dd.read_csv(f, dtype={'SimillarHTTP': 'object'})
        lind = f.rfind('/')
        slind = f.rfind('/', 0,lind-1)
        print(f[slind:])

        # Pull benign samples out and get their quantity.
        ben_df = df[df[' Label'] == 'BENIGN']

        if i == 0:
            outFrame = ben_df
            i += 1

        else:    
            outFrame = dd.concat([outFrame, ben_df])

    # Get the total count of BENIGN samples that we've aggregated.
    shben = outFrame.shape
    ben_num = shben[0].compute()
    print(f"Number of Benign samples aggregated: {ben_num}")

    # Get an upper bound on the number of attack samples to extract. Goal is to get enough to be able to make them all equal after cleaning.
    ben_num = 150000
    
    print("Beginning extraction of attack data...")

    # Second pass pulls out attack data equal in quantity to the count of benign data that we've gathered.
    for f in fileList:
        df = dd.read_csv(f, dtype={'SimillarHTTP': 'object'})
        lind = f.rfind('/')
        slind = f.rfind('/', 0,lind-1)
        print(f[slind:])
  
        # Get list of label categories
        cats = list(set(list(df[' Label'])))

        # Go through non-benign categories and add a subsample equal in quantity to the number of benign samples.
        for cat in cats:

            if cat != "BENIGN":

                print(f"Appending data for {cat}...")

                # Find our how many samples we have
                cat_df = df[df[' Label'] == cat]
                shcat = cat_df.shape
                cat_num = shcat[0].compute()

                # Get a sample if there is equal/more attack data, otherwise add everything.
                # dask only does approximate sampling, so we'll have to do some post-processing to make the numbers exactly equal.
                if cat_num > ben_num:
                    ret_df = cat_df.sample(frac = ben_num / cat_num)

                else:
                    ret_df = cat_df

                outFrame = dd.concat([outFrame, ret_df])

    # Return our aggregated DataFrame
    print(f"Aggregated df stats: {outFrame[' Label'].value_counts().compute()}\n")
    print("Extraction completed.")
    return outFrame
    


In [8]:
train = extract(fileList)
test = extract(fileList2)

Beginning extraction of benign data...
/01-12/DrDoS_DNS.csv
/01-12/DrDoS_LDAP.csv
/01-12/DrDoS_MSSQL.csv
/01-12/DrDoS_NetBIOS.csv
/01-12/DrDoS_NTP.csv
/01-12/DrDoS_SNMP.csv
/01-12/DrDoS_SSDP.csv
/01-12/DrDoS_UDP.csv
/01-12/Syn.csv
/01-12/TFTP.csv
/01-12/UDPLag.csv
Number of Benign samples aggregated: 56863
Beginning extraction of attack data...
/01-12/DrDoS_DNS.csv
Appending data for DrDoS_DNS...
/01-12/DrDoS_LDAP.csv
Appending data for DrDoS_LDAP...
/01-12/DrDoS_MSSQL.csv
Appending data for DrDoS_MSSQL...
/01-12/DrDoS_NetBIOS.csv
Appending data for DrDoS_NetBIOS...
/01-12/DrDoS_NTP.csv
Appending data for DrDoS_NTP...
/01-12/DrDoS_SNMP.csv
Appending data for DrDoS_SNMP...
/01-12/DrDoS_SSDP.csv
Appending data for DrDoS_SSDP...
/01-12/DrDoS_UDP.csv
Appending data for DrDoS_UDP...
/01-12/Syn.csv
Appending data for Syn...
/01-12/TFTP.csv
Appending data for TFTP...
/01-12/UDPLag.csv
Appending data for UDP-lag...
Appending data for WebDDoS...
Aggregated df stats: DrDoS_UDP        150001
TFTP

Save extracts, then reload them to finish processing.

In [9]:
dd.to_csv(df = train, filename = "day1_v3.csv", single_file=True)

['c:\\Users\\icarus\\Documents\\school\\Fall 2022\\CSI 5388\\dataset\\day1_v3.csv']

In [10]:
dd.to_csv(df = test, filename = "day2_v3.csv", single_file=True)

['c:\\Users\\icarus\\Documents\\school\\Fall 2022\\CSI 5388\\dataset\\day2_v3.csv']

In [36]:
df1 = pd.read_csv("day1_v3.csv")
df2 = pd.read_csv("day2_v3.csv")

C:\Users\icarus\AppData\Local\Temp\ipykernel_20472\995748964.py:1: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("day1_v3.csv")
C:\Users\icarus\AppData\Local\Temp\ipykernel_20472\995748964.py:2: DtypeWarning: Columns (86) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("day2_v3.csv")


In [37]:
print(df1[" Label"].value_counts())
print(df2[" Label"].value_counts())

DrDoS_UDP        150001
TFTP             150001
DrDoS_NetBIOS    150000
Syn              150000
UDP-lag          150000
DrDoS_LDAP       149999
DrDoS_NTP        149999
DrDoS_SSDP       149999
DrDoS_DNS        149998
DrDoS_MSSQL      149998
DrDoS_SNMP       149998
BENIGN            56863
WebDDoS             439
Name:  Label, dtype: int64
NetBIOS    300001
Syn        299999
UDP        262476
MSSQL      174391
LDAP       159929
Portmap    150000
BENIGN      56965
UDPLag       1873
Name:  Label, dtype: int64


Cleaning

In [38]:
# Remove socket information and timestamp.
drop_cols = ['Flow ID', ' Source IP', ' Source Port', ' Destination IP',' Destination Port', ' Timestamp']

df1 = df1.drop(drop_cols, axis=1)
df2 = df2.drop(drop_cols, axis=1)

# remove infinity values
df1 = df1.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

# replace invalid SimilarHTTP values with NaN
for col in df1.columns:
    if col != " Label":
        df1[col] = pd.to_numeric(df1[col], errors='coerce')

# Repeat with multiclass dataset
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

for col in df2.columns:
    if col != " Label":
        df2[col] = pd.to_numeric(df2[col], errors='coerce')

# drop remaining NaN values from both
df1 = df1.dropna(axis=0)
df2 = df2.dropna(axis=0)

In [39]:
print(df1[" Label"].value_counts())
print(df2[" Label"].value_counts())

DrDoS_SNMP       149695
DrDoS_NTP        149061
DrDoS_UDP        148071
DrDoS_SSDP       147595
DrDoS_LDAP       147318
DrDoS_MSSQL      145815
TFTP             145801
DrDoS_NetBIOS    145340
DrDoS_DNS        145091
UDP-lag          135279
Syn              130670
BENIGN            54369
WebDDoS             322
Name:  Label, dtype: int64
NetBIOS    286153
Syn        278052
UDP        258825
MSSQL      168285
LDAP       156541
Portmap    142110
BENIGN      54581
UDPLag       1873
Name:  Label, dtype: int64


We will now balance the samples and create our binary and multiclass datasets

In [40]:
# get list of labels so we can pull an equal number of samples from each. Also standardize label names.
df1[" Label"] = df1[" Label"].apply(lambda x: x.replace("DrDoS_", ""))

cols1 = df1[" Label"].unique()
print(df1[" Label"].unique())
cols2 = df2[" Label"].unique()
print(df2[" Label"].unique())

['BENIGN' 'DNS' 'LDAP' 'MSSQL' 'NetBIOS' 'NTP' 'SNMP' 'SSDP' 'UDP' 'Syn'
 'TFTP' 'UDP-lag' 'WebDDoS']
['BENIGN' 'NetBIOS' 'LDAP' 'MSSQL' 'Portmap' 'Syn' 'UDP' 'UDPLag']


In [41]:
# Assemble equal number of samples. Will be restructured in multiclass and binary datasets.
outMult = None
newAttacks = ['DNS','NTP','SNMP','SSDP','TFTP','UDP-lag']

for val in cols1:

    # too small, so we drop it
    if val == 'WebDDoS':
        continue

    elif val in newAttacks:
        slice = df1[df1[" Label"] == val].copy()
        slice = slice.sample(n=100000,random_state=42) # Ensure equal sample number
    # get all attacks of this type and cull to appropriate number
    else:
        slice = df1[df1[" Label"] == val].copy()
        slice = slice.sample(n=50000,random_state=42) # Ensure equal sample number

    # add to output
    if outMult is None:
        outMult = slice
    else:
        outMult = pd.concat([outMult, slice])

print(outMult[" Label"].value_counts())

newAttacks = ['Portmap']
for val in cols2:

    # too small, so we drop it
    if val == 'UDPLag':
        continue

    # get all attacks of this type and cull to appropriate number

    elif val in newAttacks:
        slice = df2[df2[" Label"] == val].copy()
        slice = slice.sample(n=100000,random_state=42) # Ensure equal sample number
        
    # get all attacks of this type and cull to appropriate number
    else:
        slice = df2[df2[" Label"] == val].copy()
        slice = slice.sample(n=50000,random_state=42) # Ensure equal sample number

    # add to output
    if outMult is None:
        outMult = slice
    else:
        outMult = pd.concat([outMult, slice])

print(outMult[" Label"].value_counts())

DNS        100000
NTP        100000
SNMP       100000
SSDP       100000
TFTP       100000
UDP-lag    100000
BENIGN      50000
LDAP        50000
MSSQL       50000
NetBIOS     50000
UDP         50000
Syn         50000
Name:  Label, dtype: int64
BENIGN     100000
DNS        100000
LDAP       100000
MSSQL      100000
NetBIOS    100000
NTP        100000
SNMP       100000
SSDP       100000
UDP        100000
Syn        100000
TFTP       100000
UDP-lag    100000
Portmap    100000
Name:  Label, dtype: int64


In [42]:
# Drop the benign samples to get our multiclass dataset.
multiclass = outMult[outMult[" Label"] != "BENIGN"]

print(multiclass[" Label"].value_counts())

DNS        100000
LDAP       100000
MSSQL      100000
NetBIOS    100000
NTP        100000
SNMP       100000
SSDP       100000
UDP        100000
Syn        100000
TFTP       100000
UDP-lag    100000
Portmap    100000
Name:  Label, dtype: int64


In [43]:
# binary uses all columns
binary = outMult.copy()

print(binary[" Label"].value_counts())

BENIGN     100000
DNS        100000
LDAP       100000
MSSQL      100000
NetBIOS    100000
NTP        100000
SNMP       100000
SSDP       100000
UDP        100000
Syn        100000
TFTP       100000
UDP-lag    100000
Portmap    100000
Name:  Label, dtype: int64


Binary dataset will be binarized later.

In [44]:
# get attack labels
cols = binary[" Label"].unique()

# make new attack label comprised of equal subsets of attacks.
targ = int(100000 / 12)

outBin = None

for val in cols:

    # pull category
    slice = binary[binary[" Label"] == val].copy()

    # binary is equal to subset * 11 so it's equal.
    if val == "BENIGN":
        slice = slice.sample(n=targ * 12,random_state=42) # Ensure equal sample number

    # get subset and covert to attack type.
    else:
        slice = slice.sample(n=targ,random_state=42) # Ensure equal sample number

    # add to output
    if outBin is None:
        outBin = slice
    else:
        outBin = pd.concat([outBin, slice])

print(outBin[" Label"].value_counts())

BENIGN     99996
DNS         8333
LDAP        8333
MSSQL       8333
NetBIOS     8333
NTP         8333
SNMP        8333
SSDP        8333
UDP         8333
Syn         8333
TFTP        8333
UDP-lag     8333
Portmap     8333
Name:  Label, dtype: int64


In [45]:
outBin = outBin.drop(["Unnamed: 0.1", "Unnamed: 0"], axis= 1)
outBin.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
8524,6,5727794,16,10,1317.0,6864.0,517.0,0.0,82.3125,174.906725,...,0.0,192316.0,192316.0,5535477.0,0.0,5535477.0,5535477.0,0.0,0,BENIGN
33212,17,20980,2,2,64.0,192.0,32.0,32.0,32.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN
27400,6,33009,2,2,2.0,0.0,1.0,1.0,1.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN
14313,6,128,1,2,6.0,12.0,6.0,6.0,6.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,BENIGN
40367,17,21133,2,2,84.0,116.0,42.0,42.0,42.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN


In [46]:
multiclass = multiclass.drop(["Unnamed: 0.1", "Unnamed: 0"], axis= 1)
multiclass.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
137085,17,2,2,0,1314.0,0.0,657.0,657.0,657.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DNS
115369,17,1,2,0,1878.0,0.0,939.0,939.0,939.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DNS
95102,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DNS
172547,17,47,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DNS
70556,17,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DNS


In [47]:
multiclass.to_csv("multiclass_v2.csv", index=False)
outBin.to_csv("binary_v2.csv", index=False)

In [48]:
print(outBin[" Label"].value_counts())
print(multiclass[" Label"].value_counts())

BENIGN     99996
DNS         8333
LDAP        8333
MSSQL       8333
NetBIOS     8333
NTP         8333
SNMP        8333
SSDP        8333
UDP         8333
Syn         8333
TFTP        8333
UDP-lag     8333
Portmap     8333
Name:  Label, dtype: int64
DNS        100000
LDAP       100000
MSSQL      100000
NetBIOS    100000
NTP        100000
SNMP       100000
SSDP       100000
UDP        100000
Syn        100000
TFTP       100000
UDP-lag    100000
Portmap    100000
Name:  Label, dtype: int64
